In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/
!git clone https://github.com/ultralytics/yolov5
%cd yolov5/
%pip install -qr requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 10382, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 10382 (delta 0), reused 3 (delta 0), pack-reused 10379
Receiving objects: 100% (10382/10382), 10.60 MiB | 8.93 MiB/s, done.
Resolving deltas: 100% (7167/7167), done.
/content/yolov5
     |████████████████████████████████| 596 kB 7.5 MB/s 


In [ ]:
%mkdir /content/yolov5/human/
%cd /content/yolov5/human/
!curl -L "https://public.roboflow.com/ds/eWZMd2L8xp?key=lK8AhI7CAy" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content/yolov5/human
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1296      0 --:--:-- --:--:-- --:--:--  1296
 51 6064M   51 3144M    0     0   138M      0  0:00:43  0:00:22  0:00:21 88.4M^C


In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
  with open(line,'w') as f:
    f.write(cell.format(**globals()))

### Data customize

In [ ]:

%%writetemplate /content/yolov5/1.py

import os

file_path1 = '/content/yolov5/human/valid/labels'
file_path2 = '/content/yolov5/human/valid/images'
# print(os.getcwd())
list1 = os.listdir(file_path1)
list2 = os.listdir(file_path2)
text_file_path1 = []    # 라벨 파일명
text_file_path2 = []  # 이미지 파일명

for i in list1:
    txt_file_path = file_path1 + '/' + i
    jpg_file_path = file_path2 + '/' + i[:-4] + '.jpg'

    f1 = open(txt_file_path, 'r')
    lines = f1.readlines()
    result = ''
    count = False
    for j, l in enumerate(lines):
        # ls = l.split(' ')
        if l[0:2] == '19':
            new_line = '4'+l[2:]
            result += new_line
            count = True
    if not count:
        os.remove(jpg_file_path)
        os.remove(txt_file_path)
    else:
        with open(txt_file_path, 'w') as f2:
            f2.write(result)




In [ ]:
%cd /content/yolov5/
!python 1.py

/content/yolov5


In [ ]:
###  Dataset check
from glob import glob

train_img_list = glob('/content/yolov5/human/train/images/*.jpg')
test_img_list = glob('/content/yolov5/human/valid/images/*.jpg')
valid_img_list = glob('/content/yolov5/human/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

10000 535 535


### Customize

In [ ]:
### match dataset


import yaml

with open('/content/yolov5/human/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/yolov5/human/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

with open('/content/yolov5/human/val.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

In [ ]:
%cat /content/yolov5/human/data.yaml

train: ../train/images
val: ../valid/images

nc: 80
names: ['aeroplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'cow', 'cup', 'diningtable', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorbike', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'pottedplant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'sofa', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tvmonitor', 'umbrella', 'vase', 'wine glass', 'zebra']

In [ ]:
%%writetemplate /content/yolov5/human/data.yaml

train: /content/yolov5/human/train/images
val: /content/yolov5/human/valid/images

nc: 5
names: ['person','1','2','empty','car']

In [ ]:
%cat /content/yolov5/human/data.yaml


train: /content/yolov5/human/train/images
val: /content/yolov5/human/valid/images

nc: 5
names: ['person','1','2','empty','car']

In [ ]:
with open('/content/drive/MyDrive/new/pklot/data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5l.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 

In [ ]:
%%writetemplate /content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml


# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 5  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 

### Train

In [ ]:
%time
%cd /content/drive/MyDrive/new/yolov5/
!python train.py --img 640 --batch-size 30 --epochs 20 --data /content/yolov5/human/data.yaml --cfg /content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml --weights 'yolov5l.pt' --name results --cache

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs
/content/drive/MyDrive/new/yolov5
train: weights=yolov5l.pt, cfg=/content/drive/MyDrive/new/yolov5/models/custom_yolov5l.yaml, data=/content/yolov5/human/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=30, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.

In [ ]:
!python detect.py --weights /content/drive/MyDrive/new/yolov5/runs/train/results15/weights/best.pt --img 640 --conf 0.6 --line-thickness 2 --source /content/yolov5/data/images
!python detect.py --weights /content/drive/MyDrive/new/yolov5/runs/train/results15/weights/best.pt --img 640 --conf 0.6 --line-thickness 2 --source /content/drive/MyDrive/park.mp4


detect: weights=['/content/drive/MyDrive/new/yolov5/runs/train/results15/weights/best.pt'], source=/content/yolov5/data/images, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
image 1/2 /content/yolov5/data/images/bus.jpg: 640x480 Done. (0.017s)
image 2/2 /content/yolov5/data/images/zidane.jpg: 384x640 Done. (0.017s)
Speed: 0.5ms pre-process, 16.8ms inference, 0.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp51
detect: weights=['/content/drive/MyDrive/new/yolov5/runs/train/result

In [ ]:
%cd /content/yolov5/
!python detect.py --weights 'yolov5l.pt' --img 640 --conf 0.6 --line-thickness 2 --source /content/drive/MyDrive/new/camera2/test/images/

/content/yolov5
detect: weights=['yolov5l.pt'], source=/content/drive/MyDrive/new/camera2/test/images/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-163-gd95978a torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

100% 89.2M/89.2M [00:04<00:00, 21.7MB/s]

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
image 1/81 /content/drive/MyDrive/new/camera2/test/images/2016-01-14_0804.jpg: 480x640 2 cars, Done. (0.024s)
image 2/81 /content/drive/MyDrive/new/camera2/test/images/2016-01-14_0834.jpg: 480x640 1 person, 4 cars, Done. (0.025s)
image 3/81 /content/drive/MyDrive/new/camera2/test/images/2016-01-14_0904.jpg: 

### Person model + 2camera 

In [ ]:
!python detect.py --weights /content/drive/MyDrive/models/person_best.pt /content/drive/MyDrive/models/camera8.pt --img 640 --conf 0.6 --line-thickness 2 --source /content/drive/MyDrive/park.mp4 

detect: weights=['/content/drive/MyDrive/new/yolov5/runs/train/results11/weights/best.pt', '/content/drive/MyDrive/models/camera8.pt'], source=/content/drive/MyDrive/park.mp4, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2021-12-23 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
Fusing layers... 
Model Summary: 367 layers, 46129818 parameters, 0 gradients, 107.9 GFLOPs
Ensemble created with ['/content/drive/MyDrive/new/yolov5/runs/train/results11/weights/best.pt', '/content/drive/MyDrive/models/camera8.pt']

video 1/1 (1/4743) /content/drive/MyDrive/park.mp4: 384x640 22

In [ ]:
!pwd

/content/drive/My Drive/new/yolov5
